#### 사용자 정의 모델 클래스
 - 부모클래스 : nn.Module
 - 필수오버라이딩
    * __init__() : 모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

In [28]:
## 모듈 로딩
import torch                                # 텐서 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화함수 등등 )
import torch.optim as optim                 # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈
import torchmetrics as metrics              
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈

 - [기본] 신경망클래스 <hr>
    * 입력층 - 입력 : 피쳐 고정
    * 출력층 - 출력 : 타겟 수 고정
    * 은닉층 - 고정

In [29]:
# 모델 설계
# 데이터셋 : 피쳐 4개, 타겟 1개, 회귀
# 입 력 층 : 입력 004개  출력 020개    AF ReLU  (s/ 시그모이드 써도 괜찮다. ReLU를 쓰든 시그모이드를 쓰던 그건 자율적. 뭔지 모르겟음 공부 필요)
# 은 닉 층 : 입력 020개  출력 100개    AF ReLU
# 출 력 층 : 입력 100개  출력 001개    AF Sigmoid & Softmax

# 2진 분류에는 시그모이드, 다중분류에는 소프트맥스
# 회귀는 필요없다. X

In [30]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
class MyModel(nn.Module):
    def __init__(self):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스 생성
        self.input_layer = nn.Linear(1,20)      # w 4 + b 1  => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20,100)   # w 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100,1)    # w 100 + b 1 => 1P, 101

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 CallBack Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # y = x1w1 + x2w2 + x3w3 + x4w4 + b
        F.relu(y)                   # 0 <= y    ---> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y)    # y = x1w1 + x2w2 + ... + x20w20 + b
        F.relu(y) 

        # 

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b

In [32]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수 동적인 모델 
class MyModel2(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 : 시스템에서 호출되는 함수)

    def __init__(self, in_features,  in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스 생성
        self.input_layer = nn.Linear(in_features,in_out)      # w 4 + b 1  => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(in_out,h_out)   # w 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(h_out,1)    # w 100 + b 1 => 1P, 101

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 CallBack Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # y = x1w1 + x2w2 + x3w3 + x4w4 + b
        F.relu(y)                   # 0 <= y    ---> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y)    # y = x1w1 + x2w2 + ... + x20w20 + b
        F.relu(y) 

        # 

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b

In [46]:
# 은닉층의 개수가 동적인 모델

class MyModel4(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 : 시스템에서 호출되는 함수)

    def __init__(self, in_features, Linear ,in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스 생성
        self.input_layer = nn.Linear(in_features,in_out)      # w 4 + b 1  => 1P, 5 * 20 = 100개 변수
        for i in range(Linear):
            self.hidden_layer = nn.Linear(in_out//Linear,h_out)
            self.hidden_layer = nn.Linear(in_out,h_out)
        self.output_layer = nn.Linear(h_out,1)    # w 100 + b 1 => 1P, 101

        

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 CallBack Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # y = x1w1 + x2w2 + x3w3 + x4w4 + b
        F.relu(y)                   # 0 <= y    ---> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y)    # y = x1w1 + x2w2 + ... + x20w20 + b
        F.relu(y) 

        # 

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b

In [47]:
## 모델 인스턴스 생성
m1 = MyModel()

m2 = MyModel2(3,100,70)
m2

m4 = MyModel4(2,4,100,70)
m4

MyModel4(
  (input_layer): Linear(in_features=2, out_features=100, bias=True)
  (hidden_layer): Linear(in_features=25, out_features=70, bias=True)
  (output_layer): Linear(in_features=70, out_features=1, bias=True)
)

In [34]:
for m in m1.parameters(): print(m)

Parameter containing:
tensor([[ 0.5153],
        [-0.4414],
        [-0.1939],
        [ 0.4694],
        [-0.9414],
        [ 0.5997],
        [-0.2057],
        [ 0.5087],
        [ 0.1390],
        [-0.1224],
        [ 0.2774],
        [ 0.0493],
        [ 0.3652],
        [-0.3897],
        [-0.0729],
        [-0.0900],
        [ 0.1449],
        [-0.0040],
        [ 0.8742],
        [ 0.3112]], requires_grad=True)
Parameter containing:
tensor([-0.3724, -0.6040, -0.1676, -0.4313, -0.3204,  0.0479,  0.5961,  0.5435,
        -0.9776,  0.6199,  0.2794,  0.9486,  0.6601, -0.9111, -0.9508, -0.4823,
         0.8781, -0.1666,  0.4280, -0.4647], requires_grad=True)
Parameter containing:
tensor([[ 0.2194, -0.0946,  0.1677,  ..., -0.1493,  0.1362,  0.0694],
        [-0.1445,  0.1452,  0.1358,  ...,  0.1108, -0.2071,  0.1126],
        [-0.1572, -0.1687,  0.0136,  ...,  0.0401, -0.0951, -0.0677],
        ...,
        [-0.2228,  0.0137,  0.0685,  ...,  0.0825, -0.1046, -0.1153],
        [-0.041

In [35]:
# 학습 진행 ===> 모델 인스턴스명()
# 임의의 데이터
dataTS = torch.FloatTensor([[1,3,5,7],[2,4,6,8]])
targetTS = torch.FloatTensor([[4],[5]])

# 학습
pre_y = m1(dataTS)

print(pre_y)


calling forward()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x4 and 1x20)